In [128]:
import csv
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
qa_pd = pd.DataFrame(columns = ['title','selftext','score','source'])
non_qa_pd = pd.DataFrame(columns = ['title','selftext','score','source'])

In [129]:
badwords = ['nigger', 'fuck', 'sex', 'negro', 'cunt', 'bitch', 'cum', 'gay', 'dyke', 'fag', 'faggot']
redditwords = ['Edit:', 'edit:']


def check_bad_word(badwordlist, sentence):
    sentencecopy = sentence.split(' ')
    for word in badwords:
        if word in sentencecopy:
            return False
    return True


In [131]:
def filter_csv2(data, source,length_limit=250):
    is_good_list = []
    global qa_pd
    global non_qa_pd
    for i in range(len(data)):
        if pd.isnull(data.iloc[i]['title']):
            title = ''
        else:
            title = data.iloc[i]['title']

        if pd.isnull(data.iloc[i]['selftext']):
            body = ''
        else :
            body = data.iloc[i]['selftext']

        data.at[i,'source'] = source
        sentence = title + body
        if (len(sentence) == 0):
            continue
        data.at[i,'title'] = title.replace('\n','')
        data.at[i,'selftext'] = body.replace('\n','')
        for word in redditwords:
            if word in body:
                data.at[i,'selftext'] = body[:body.index(word)]
                
                
        #Good if data does not contain bad words and the length of the sentence is between 2 and 250 and not duplicated
        is_good = check_bad_word(badwords, sentence) and (len(sentence) > 2) and (len(sentence) < length_limit) 

        #check if the data is removed
        if type(body) is str and ( body == "[deleted]" or not body=="[removed]" ):
            is_good = !is_good
            
        #check if it is a question
        is_qa = is_good and title.count('?') == 1
        
        if title in qa_pd["title"] or title in non_qa_pd["title"]:
            continue
        
        if is_good:
            if is_qa:
                qa_pd = qa_pd.append(data.loc[i,:])
            else:
                non_qa_pd=non_qa_pd.append(data.loc[i,:])


In [132]:
def write_csv(posts, name):
    name = name + ".csv"
    posts.to_csv(name, index=False)

In [133]:
def read_in_csv(csv_file):
    if '.csv' in csv_file:
        data = pd.read_csv(csv_file)
    elif '.tsv' in csv_file:
        data = pd.read_csv(csv_file,sep="\t")
    return data

In [134]:
files = ["puns.csv", "AntiAntiJokes.csv", "rjokes.csv"]

In [135]:
pd_list = []

#Read in csv files

for file in files:
    fname = "../raw/" + file
    df = read_in_csv(fname)
    if (df.columns[0] != 'title'):
        col_data = df['score']
        df = df.drop(["score"], axis=1)
        df["score"] = col_data
        df["source"] = None
    pd_list.append(df)

In [136]:
for i in range(len(pd_list)):
    filter_csv2(pd_list[i], files[i])

In [139]:
write_csv(qa_pd, "qa_total")

In [140]:
write_csv(non_qa_pd,"non_qa_total")